In [1]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=5, precision=2, linewidth=75)

In [2]:
import csv
wine_path = "/home/bartek/dlwpt-code-master/data/p1ch4/tabular-wine/winequality-white.csv"
wine_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",skiprows=1)

In [3]:
col_list = next(csv.reader(open(wine_path),delimiter=";"))

In [4]:
wineq=torch.from_numpy(wine_numpy)

In [5]:
data=wineq[:,:-1]

In [6]:
target=wineq[:,-1]

In [7]:
target=target.long()

In [8]:
target_onehot=torch.zeros(target.shape[0],10)

In [9]:
target=target.unsqueeze(1)

In [10]:
target_onehot.scatter_(1,target,1)

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [11]:
data_mean=torch.mean(data,dim=0)

In [12]:
data_var=torch.var(data,dim=0)

In [13]:
data_normalized=(data-data_mean)/torch.sqrt(data_var)

In [14]:
data_normalized.shape

torch.Size([4898, 11])

In [15]:
x = data_normalized.clone().detach().requires_grad_(True)

In [16]:
x = x.to(device='cuda')

In [17]:
y = target_onehot.clone().detach().requires_grad_(True)

In [18]:
y = y.to(device = 'cuda')

In [19]:
from torch.utils.data import DataLoader, TensorDataset, random_split
dataset = TensorDataset(x, y)

In [20]:
train_ds, val_ds = random_split(dataset,[3000,1898])
batch_size=50
train_loader = DataLoader(train_ds,batch_size,shuffle=True)
val_loader = DataLoader(train_ds,batch_size)

In [63]:
import torch.nn as nn
import torch.nn.functional as F

In [958]:
class Net(nn.Module):
    def __init__(self):
        
        self.device = torch.device("cuda")
        super(Net, self).__init__()
        self.input_linear=nn.Linear(11,25)
        self.midle_linear1=nn.Linear(25,50)
#         self.midle_linear2=nn.Linear(100,100)
#         self.midle_linear3=nn.Linear(100,100)
#         self.midle_linear4=nn.Linear(100,100)
#         self.midle_linear5=nn.Linear(100,100)
#         self.midle_linear6=nn.Linear(100,100)        
#         self.midle_linear7=nn.Linear(100,100)        
        self.midle_linear9=nn.Linear(50,25)
        self.output_linear=nn.Linear(25,10)
    
    def forward(self, x):
        h_relu = self.input_linear(x)
        h_relu = torch.relu(self.midle_linear1(h_relu))
#         h_relu = self.midle_linear2(h_relu)
#         h_relu = self.midle_linear3(h_relu)
#         h_relu = self.midle_linear4(h_relu)
#         h_relu = self.midle_linear5(h_relu)
#         h_relu = self.midle_linear6(h_relu)
#         h_relu = self.midle_linear7(h_relu)
        h_relu = torch.relu(self.midle_linear9(h_relu))
        y_pred = self.output_linear(h_relu)
#         h_relu = torch.relu(self.input_linear(x))
#         h_relu = torch.relu(self.midle_linear1(h_relu))
#         h_relu = torch.relu(self.midle_linear2(h_relu))
#         h_relu = torch.relu(self.midle_linear3(h_relu))
#         h_relu = torch.relu(self.midle_linear4(h_relu))
#         y_pred = self.output_linear(h_relu)

        return y_pred

In [959]:
model = Net().to(device="cuda")
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [960]:
for t in range(100):
    y_pred = model(x)
    loss = criterion(y_pred,y)
    if t%10 == 9:
        print(t, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

9 3190.336181640625
19 3019.03271484375
29 2944.467041015625
39 2919.28076171875
49 2991.9296875
59 2893.0966796875
69 2939.545166015625
79 2898.586669921875
89 2889.372314453125
99 2887.887939453125


In [961]:
wynik = torch.argmax(y_pred, 1).unsqueeze(1)

In [962]:
target = target.to(device='cuda')

In [963]:
import math
roznica=wynik-target

In [964]:
blad=sum(torch.abs(roznica))

In [965]:
blad

tensor([2509], device='cuda:0')

In [966]:
zero, jeden, dwa, trzy, cztery, _jeden, _dwa, _trzy, _cztery = 0,0,0,0,0,0,0,0,0

In [967]:
for i in range(4898):
    if roznica1[i]==0 :  zero += 1
    elif roznica1[i]==1:  jeden += 1
    elif roznica1[i]==2:  dwa += 1
    elif roznica1[i]==3:  trzy += 1
    elif roznica1[i]==4:  cztery += 1
    elif roznica1[i]==-3:  _cztery += 1        
    elif roznica1[i]==-1: _jeden += 1
    elif roznica1[i]==-2: _dwa += 1
    elif roznica1[i]==-3: _trzy += 1

In [968]:
print(cztery,trzy,dwa,jeden,zero, _jeden, _dwa, _trzy,_cztery)

5 20 110 1137 2610 907 107 0 2


In [969]:
zero+ jeden+ dwa+ trzy+ _jeden+ _dwa+ _trzy+_cztery+cztery

4898

In [970]:
#dodać batch
print(wynik.shape,target.shape)

torch.Size([4898, 1]) torch.Size([4898, 1])


In [971]:
third_tensor = torch.cat((wynik, target), 1)

In [972]:
third_tensor = third_tensor.to(device = 'cpu')

In [973]:
import torch
import pandas as pd
import numpy as np

third_tensor_np = third_tensor.numpy()
third_tensor_df = pd.DataFrame(third_tensor_np)
third_tensor_df.columns=["wynik","target"]
third_tensor_df.to_csv('/home/bartek/third_tensor.csv')
